function to get top 5 categories from each department

In [1]:
def top_5_categories(trx, item):
    """
    Identify the top 5 categories within each department based on total sales quantity.

    Args:
    - trx: DataFrame, the transaction data containing item codes and sales quantities
    - item: DataFrame, the item data containing item codes and category descriptions

    Returns:
    - DataFrame: A DataFrame containing the top 5 categories within each department based on total sales quantity
    """
    # Join transaction data with item data on stg_item_cd
    top_categories = (
        trx
        .join(
            item,
            on="stg_item_cd",
            how="inner"
        )
        .groupBy(
            "stg_item_dept_desc_txt", "stg_item_category_desc_txt"
        )
        .agg(
            f.sum("sales_qty").alias("total_sales_qty")
        )
    )

    # Define window specification for ranking within each department
    window_spec = Window.partitionBy("stg_item_dept_desc_txt").orderBy(f.col("total_sales_qty").desc())

    # Add rank column based on total_sales_qty within each department
    top_categories = top_categories.withColumn("rank", f.row_number().over(window_spec))

    # Filter top 5 categories within each department
    top_5 = top_categories.filter(f.col("rank") <= 5).drop("rank")

    return top_5



To get the weekly sales for top 5 categories in each department

In [1]:
def weekly_sales_df(trx, item_info) :
    """
    Calculate the weekly sales quantities for the top 5 categories within each department.

    Args:
    - trx: DataFrame, the transaction data containing item codes, sales quantities, and dates
    - item_info: DataFrame, the item data containing item codes and category descriptions

    Returns:
    - DataFrame: A DataFrame containing the weekly sales quantities for the top 5 categories within each department
    """
    # Identify the top 5 categories within each department
    top_5_cats = top_5_categories(trx, item_info)

    # Compute weekly sales for top 5 categories
    weekly_sales = (
        trx
        .join(
            item_info,
            on="stg_item_cd",
            how="inner"
        )
        .filter(f.col("stg_item_category_desc_txt").isin([row.stg_item_category_desc_txt for row in top_5_cats.select("stg_item_category_desc_txt").collect()]))
        .groupBy("stg_outlet_cd", "week", "stg_item_category_desc_txt")
        .agg(f.sum("sales_qty").alias("weekly_sales_qty"))
        .orderBy("stg_outlet_cd", "stg_item_category_desc_txt", "week")
    )

    return weekly_sales

function to get final categories

In [ ]:
def get_final_categories(transaction, item_info): 
    """
    Given transaction and item_info DataFrames, returns the distinct final top 5 categories in each department.

    Args:
        transaction (DataFrame): DataFrame containing transaction data.
        item_info (DataFrame): DataFrame containing item information.

    Returns:
        DataFrame: A DataFrame with distinct final categories.
    """
    final_top_5_category = top_5_categories(transaction, item_info)
    final_categories = final_top_5_category.select("stg_item_category_desc_txt").distinct()
    return final_categories